In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('preprocessedData.csv')

In [3]:
data.head()

,duration,day_of_week,time_of_day,start_station_id_distance,end_station_id_distance,temp_category,mean_dew_point_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover
0,608,1,noon,0.274358,0.279310,less,45.0,69.0,9.0,1.0,0,5.0
1,387,1,rush_hour,0.274358,0.273022,between,48.0,67.0,10.0,9.0,T,7.0
2,1207,1,rush_hour,0.274358,0.287692,between,48.0,67.0,10.0,9.0,T,7.0
3,662,1,noon,0.274358,0.275123,less,45.0,69.0,9.0,1.0,0,5.0
4,343,1,rush_hour,0.274358,0.273022,less,44.0,64.0,9.0,4.0,0,3.0


In [7]:
y = pd.DataFrame(data['duration'], columns=['duration'])

In [9]:
x = data.drop(columns=['duration'])

In [10]:
x.head()

,day_of_week,time_of_day,start_station_id_distance,end_station_id_distance,temp_category,mean_dew_point_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover
0,1,noon,0.274358,0.279310,less,45.0,69.0,9.0,1.0,0,5.0
1,1,rush_hour,0.274358,0.273022,between,48.0,67.0,10.0,9.0,T,7.0
2,1,rush_hour,0.274358,0.287692,between,48.0,67.0,10.0,9.0,T,7.0
3,1,noon,0.274358,0.275123,less,45.0,69.0,9.0,1.0,0,5.0
4,1,rush_hour,0.274358,0.273022,less,44.0,64.0,9.0,4.0,0,3.0


In [11]:
y.head()

,duration
0,608
1,387
2,1207
3,662
4,343
